In [4]:
# monte carlo estimate of pi
# consider a circle inside a 1x1 square, where the 4 sides of the cirls exactly 
# touch the sides of the square
# then area(circle) / area(square) = pi * r ^ 2 / 1
#                                  = pi * .5 ^ 2
# but area(circle) / area(square) ~ ratio achieved by randomly
# selecting points inside square and taking point inside circle over total 
# selected points
# thus pi ~ ratio / .5 ^ 2

import random

def insideCircle(x, y):
    #is the point (x,y) inside the circle of radius .5 centered at (.5,.5)
    return (x - .5)**2 + (y - .5)**2 <= .5**2

def piEstimate(numSim):
    inCircle = 0
    for x in range(numSim):
        r1 = random.random()
        r2 = random.random()
        if(insideCircle(r1, r2)):
            inCircle += 1
    ratio = inCircle / numSim
    return ratio / (.5**2)
        

print(piEstimate(1000000))

3.14148


In [5]:
# now we do a financial example
# we use the Monte Carlo method to estimate a call option price
# making all the standard Black-Scholes assumptions
# we estimate the call option price by calculating scenario paths
# where the underlying process is geometric Brownian motion, consistent
# with the assumptions of the Black-Scholes model

import math 
from scipy.stats import norm
import scipy.optimize as opt
import numpy as np
import statistics as stats

tradeDaysPerYear = 252

# implement standard Black-Scholes functions, to compare against
# the monte carlo based prices

def nPrime(x):
    d = math.exp(-x ** 2 / 2) / math.sqrt(2 * math.pi)
    return d

def d1(vol, t, T, r, S_t, K):
    f1 = 1 / (vol * math.sqrt(T - t))
    f2 = math.log(S_t / K) + (r + (vol * vol) / 2)  * (T-t)
    return f1 * f2

def d2(vol, t, T, r, S_t, K):
    return d1(vol, t, T, r, S_t, K) - vol * math.sqrt(T - t)

def PV(t, T, r, K):
    return K * math.exp(-r * (T - t))

def bsCallPrice(vol, t, T, r, S_t, K):
    if(t >= T):
        return max(0, S_t - K)
    d_1 = d1(vol, t, T, r, S_t, K)
    d_2 = d2(vol, t, T, r, S_t, K)
    price = norm.cdf(d_1) * S_t - norm.cdf(d_2) * PV(t, T, r, K)
    return price

class monteCarloPrice:
    
    def __init__(self, t, T, r, S_t, K, realizedVol, numScenarios, stepsPerDay):
        self.startingPrice = S_t
        self.realizedVol = realizedVol
        self.t = t
        self.T = T
        self.r = r
        self.S_t = S_t
        self.K = K
        self.finalPriceSeries = []
        self.numScen = numScenarios
        self.stepsPerDay = stepsPerDay
        days = (self.T - self.t) * tradeDaysPerYear
        self.steps = round(days * self.stepsPerDay)
        self.stepLen = (self.T - self.t) / self.steps

    def nextStep(self):
        volMovement =  self.realizedVol * np.random.normal(0,1) * math.sqrt(self.stepLen) * self.S_t    
        driftMovement = self.stepLen * r * self.S_t
        self.move = driftMovement + volMovement
        self.S_t = self.S_t + self.move

    def runSimulation(self):
        self.S_t = self.startingPrice #reset S_t to original starting price before running next simulation
        for x in range(self.steps):
            self.nextStep()
        return self.S_t #return final price of simulation

    def price(self):
        for x in range(self.numScen):
            finalPrice = self.runSimulation()
            self.finalPriceSeries.append(finalPrice)
        payouts = [max(0, x - self.K) for x in self.finalPriceSeries]
        avgPayout = stats.mean(payouts)
        discountedAvgPayout = math.exp(-(self.T - self.t) * self.r) * avgPayout
        return discountedAvgPayout

In [6]:
# next we estimate the price of a certain call option with the 
# monte carlo method (calculating the average discounted payout
# under the risk neutral measure)
# we calculate estimates given a certain number of scenario paths
# and observe how the distribution of price estimates 
# behaves according to the number of scenario paths
# we observe roughly a halving of the standard error 
# by quadrupling the number of scenarios, which is expected and 
# is a standard result of statistics

vol = .1
r = .05
t = 0
T = 1.0
S = 100
K = 100

exactCallPrice = bsCallPrice(vol, t, T, r, S, K)
print('exact price: ', exactCallPrice)

numScenarios = 10
neededForHalvingStError = 4 #need 4x more scenarios to halve standard error
stepsPerDay = 1

monteCarlo = monteCarloPrice(t, T, r, S, K, vol, numScenarios, stepsPerDay)
monteCarloPrices1 = [monteCarlo.price() for x in range(1000)]
print('average monte carlo price from 10 scenario path simulations of 1000 simulations: ', stats.mean(monteCarloPrices1))
print('standard error from 10 scenario path simulations: ', stats.stdev(monteCarloPrices1))

numScenarios = numScenarios * neededForHalvingStError
monteCarlo2 = monteCarloPrice(t, T, r, S, K, vol, numScenarios, stepsPerDay)
monteCarloPrices2 = [monteCarlo2.price() for x in range(1000)]
print('average monte carlo price from 40 scenario path simulation of 1000 simulations: ', stats.mean(monteCarloPrices2))
print('standard error from 40 scenario path simulations: ', stats.stdev(monteCarloPrices2))

numScenarios = numScenarios * neededForHalvingStError
monteCarlo3 = monteCarloPrice(t, T, r, S, K, vol, numScenarios, stepsPerDay)
monteCarloPrices3 = [monteCarlo3.price() for x in range(1000)]
print('average monte carlo price from 160 scenario path simulation of 1000 simulations: ', stats.mean(monteCarloPrices3))
print('standard error from 160 scenario path simulations: ', stats.stdev(monteCarloPrices3))

exact price:  6.804957708822144
average monte carlo price from 10 scenario path simulations of 1000 simulations:  6.898543360495029
standard error from 10 scenario path simulations:  0.10900295992137861
average monte carlo price from 40 scenario path simulation of 1000 simulations:  6.858748139592203
standard error from 40 scenario path simulations:  0.06871819672699082
average monte carlo price from 160 scenario path simulation of 1000 simulations:  6.790119518735177
standard error from 160 scenario path simulations:  0.029431638915975823
